In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pandas as pd
import plotly.graph_objects as go
# from ipywidgets import interact, widgets
import ipywidgets as widgets
widgets.IntSlider()
from IPython.display import display, clear_output

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
# Tạo khung thời gian
from datetime import datetime, timedelta, date
from_year = 2024
to_year = 2024
from_month = 8
to_month = 8
from_date = 5
to_date = 11
def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(days=day) for day in range(delta.days + 1)]
    return days
start = date(from_year, from_month, from_date)
end = date(to_year, to_month, to_date)
from_to_date = date_range(start, end)
from_to_date = set(from_to_date)

In [3]:
base_path =  'freport_w6/test_oos'
list_report =  sorted(os.listdir(base_path))

FileNotFoundError: [Errno 2] No such file or directory: 'freport_w6/test_oos'

# 1. GPS
## Dữ liệu check in shop

In [ ]:

def read_gps(list_report):
    list_gps = []
    for report in list_report:
        if 'GPS' in report:
            file_path = os.path.join(base_path, report)
            df_gps = pd.read_excel(file_path, skiprows=0)
            try:
                # Bởi vì dòng số 1 trong file dữ liệu GPS không có dữ liệu nên bỏ đi
                df_gps = df_gps.drop(df_gps.index[0])
            except IndexError:
                pass
            list_gps.append(df_gps)
    df_gps = pd.concat(list_gps, ignore_index=True)

    # Chuyển đổi tên cột Date sang Day
    df_gps['Day'] = df_gps['Date']
    # Tạo cột Apple ID
    df_gps['Apple ID'] = df_gps['Apple Store ID']
    # Tạo cột Date để check ngày
    df_gps['Date'] = pd.to_datetime(df_gps[['Year', 'Month', 'Day']].astype('int'))
    # Xếp lại dữ liệu theo ngày nào mới nhất đưa lên đầu
    df_gps.sort_values('Date', inplace=True, ascending=False)
    # loại bỏ dữ liệu bị duplicate
    df_gps.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    # Bỏ qua các lượt checkin Apple Moment
    df_gps = df_gps[df_gps['Channel'] != 'A.M']
    # Chuyển đổi dữ liệu sai lệch check in sang định dạng int
    df_gps['Sai lệch'] = df_gps['Sai lệch'].map(lambda x: int(re.sub('[a-z\s,]', '', x)))
    
    # df_gps.reset_index(inplace=True)
    return df_gps
    
df_gps = read_gps(list_report)

In [ ]:
# Xem dữ liệu chỉ trong khung thời gian đã xác định để phân tích - Lưu ý là dữ liệu sẽ thay đổi inplace
df_gps = df_gps[df_gps['Date'].isin(from_to_date)]


In [ ]:
# Có 3 feature để đưa ra phân tích: 'Staff Name', 'Channel', 'POS Type'

# Plotting

feature_gps =  'POS Type'

plt.figure(figsize=(8, 2.5))
df_gps[feature_gps].value_counts().plot(kind='barh', color='skyblue')
plt.xlabel('Số lượt vist')
plt.ylabel('')
plt.title(f'Biểu đồ số lượt visit tính theo {feature_gps}')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest count on top

# Add grid
plt.grid(axis='x', linestyle='--', linewidth=0.7, color='grey')  # Customize grid appearance

plt.show()

# ASM đã visit bao nhiêu cửa hàng trong giai đoạn này
df_gps[feature_gps].value_counts()

In [ ]:
# Những dữ liệu check in bị quá xa so với shop cần xem xét
df_gps[(df_gps['Sai lệch'] > 1000) | (df_gps['Sai lệch'] < 0)]

# 2. Daily-Sell-out

In [ ]:

def read_sell_out(list_report):
    list_daily_sell = []
    for report in list_report:
        if 'Daily-Sell-out' in report:
            file_path = os.path.join(base_path, report)
            df_sell = pd.read_excel(file_path, sheet_name='Daily Sell - out', skiprows=1)
            try:
                df_sell = df_sell.drop(df_sell.index[0])
            except IndexError:
                pass
            list_daily_sell.append(df_sell)
    df_sell = pd.concat(list_daily_sell, ignore_index=True)

    # Chuyển đổi Date thành Day
    df_sell['Day'] = df_sell['Date']
    # Tạo cột Date để lọc ngày
    df_sell['Date'] = pd.to_datetime(df_sell[['Year', 'Month', 'Day']].astype('int'))
    df_sell.sort_values('Date', inplace=True, ascending=False)
    # loại bỏ dữ liệu trùng lặp
    df_sell.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    
    return df_sell
df_sell = read_sell_out(list_report)

In [ ]:
# Xem dữ liệu chỉ trong khung thời gian đã xác định để phân tích - Lưu ý là dữ liệu sẽ thay đổi inplace
df_sell = df_sell[df_sell['Date'].isin(from_to_date)]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output

# Assuming df_sell is your DataFrame

def create_plot(feature):
    plt.figure(figsize=(10, 6))
    df_sell[feature].value_counts().plot(kind='barh', color='skyblue')
    plt.xlabel('Số lượng')
    plt.ylabel('')
    plt.title(f'Biểu đồ số sales out do ASM hỗ trợ - {feature}')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', linestyle='--', linewidth=0.7, color='grey')
    plt.tight_layout()
    plt.show()
    
    # Display value counts
    display(df_sell[feature].value_counts().reset_index().rename(columns={'index': feature, feature: 'Count'}))

def on_button_clicked(b):
    clear_output(wait=True)
    feature = b.description
    create_plot([feature])

# Create buttons
buttons = [widgets.Button(description=feature) for feature in ['Age', 'Job', 'Reason', 'Payment Method', 'Attach']]

def on_button_clicked(b):
    print(f"Button clicked: {b.description}")  # Debug print
    clear_output(wait=True)
    feature = b.description
    create_plot(feature)
    display(widgets.HBox(buttons))

# Display buttons
display(widgets.HBox(buttons))

# Initial plot (you can choose which feature to show initially)
create_plot('Age')

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension

# !pip install ipywidgets --upgrade
# !pip install notebook --upgrade

!jupyter notebook stop
!jupyter notebook


In [ ]:
import ipywidgets as widgets
from IPython.display import display

def on_button_click(b):
    print(f"Button clicked: {b.description}")

# Create a button and attach the click event
button = widgets.Button(description="Click Me")
button.on_click(on_button_click)
display(button)


In [ ]:
# Có một số câu hỏi có thể đặt ra, điền vào feature bên dưới để ra kết quả

# 'STAFF NAME', 'Product', ['Model', 'Memory', 'Color'], 'Customer', 'Gender',
# 'Age', 'Job', 'Reason', 'Payment Method', 'Attach' 

# Chỉ cần chọn feature là ra biểu đồ để phân tích sâu thêm
feature =  ['Age']

# Plotting
plt.figure(figsize=(8, 2.5))
df_sell[feature].value_counts().plot(kind='barh', color='skyblue')
plt.xlabel('Số lượng')
plt.ylabel('')
plt.title('Biểu đồ số sales out do ASM hỗ trợ')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest count on top

# Add grid
plt.grid(axis='x', linestyle='--', linewidth=0.7, color='grey')  # Customize grid appearance

plt.show()

# Ai là best seller trên F-Track
df_sell[feature].value_counts().reset_index()

In [ ]:
# sell_columns =  df_sell.columns.tolist()
# def list_unique_categories(df):
#     categorical_cols = df.select_dtypes(include=['category', 'object']).columns
#     unique_values = {}
    
#     for col in categorical_cols:
#         unique_values[col] = df[col].unique().tolist()
    
#     return unique_values

# unique_categories = list_unique_categories(df_sell)
# print("Unique categories for categorical columns:", unique_categories)

# 3. Note

In [ ]:

def read_daily_note(list_report):
    list_daily_note = []
    for report in list_report:
        if 'Note' in report:
            file_path = os.path.join(base_path, report)
            df_note = pd.read_excel(file_path, skiprows=1)
            list_daily_note.append(df_note)
    df_note = pd.concat(list_daily_note, ignore_index=True)
    # Loại bỏ dữ liệu Apple Moment
    df_note = df_note[df_note['Channel'] != 'A.M']
    # Tạo cột Date để check khung thời gian
    df_note['Date'] = pd.to_datetime(df_note[['Year', 'Month', 'Day']].astype('int'))
    # Sắp xếp dữ liệu theo trình tự ngày gần nhất đưa lên đầu
    df_note.sort_values('Date', inplace=True, ascending=False)
    # loại bỏ dữ liệu trùng lắp nếu có, giữ lại ngày gần nhất
    df_note.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    return df_note


df_note = read_daily_note(list_report)

In [ ]:
# Xem dữ liệu chỉ trong khung thời gian đã xác định để phân tích - Lưu ý là dữ liệu sẽ thay đổi inplace
df_note = df_note[df_note['Date'].isin(from_to_date)]

In [ ]:
# Kiểm tra xem những visit nào bị quên note trong summary về thị trường
if len(df_gps) != len(df_note):
    # Merge DataFrames with indicator=True
    merged_df = df_gps.merge(df_note, on=['Staff Name', 'Apple ID', 'Channel', 'Store Name', 'Date'], 
                          how='left', indicator=True)
    
    # Filter rows that are only in df1 but not in df2
    df_gps_only = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
    df_gps_only =  df_gps_only[['Staff Name', 'Apple ID', 'Channel', 'Store Name', 'Date']]
df_gps_only.reset_index()

# 3. OOS

In [ ]:
# Lưu ý: dữ liệu stock trên hệ thống ghi đè, ngày mới nhất sẽ hiển thị, dữ liệu cũ sẽ bị xoá khỏi hệ thống trong tháng
# đó (nếu đổ dữ liệu theo tháng)
# Nếu muốn tra cứu dữ liệu stock thì cần đổ dữ liệu theo tuần

def read_oos(list_report):
    list_daily_oos = []
    for report in list_report:
        if 'OOS' in report:
            file_path = os.path.join(base_path, report)
    
            # Read the first 4 rows to handle headers manually
            df_header = pd.read_excel(file_path, header=None, nrows=4)
            header_1_16 = df_header.iloc[1, :16].tolist()
            header_17_20 = df_header.iloc[2, 16:20].tolist()
            header_21_80 = df_header.iloc[3, 20:81].tolist()
            header = header_1_16 + header_17_20 + header_21_80
            
            # Read the full data, skipping the first 4 rows (headers)
            df_oos = pd.read_excel(file_path, skiprows=3)
            df_oos = df_oos.iloc[:, :81]
            df_oos.columns = header
            list_daily_oos.append(df_oos)
    df_oos = pd.concat(list_daily_oos, ignore_index=True)
    # Tạo cột Date để lọc ngày
    df_oos['Date'] = pd.to_datetime(df_oos[['Year', 'Month', 'Day']].astype('int'))
    # Xếp lại dữ liệu theo trình tự thời gian cũ tới mới
    df_oos.sort_values('Date', inplace=True, ascending=False)
    # Loại bỏ dữ liệu lặp lại nếu có gây ra bởi việc download nhầm nhiều bảng dữ liệu
    df_oos.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    # Move 'Date' to the 2nd column position
    cols = df_oos.columns.tolist()  # Get current column order
    cols.remove('Date')  # Remove 'Date' from the list
    cols.insert(1, 'Date')  # Insert 'Date' at the 2nd position (index 1)
    df_oos = df_oos[cols]
    return df_oos


df_oos = read_oos(list_report)

In [ ]:
# Xem dữ liệu chỉ trong khung thời gian đã xác định để phân tích - Lưu ý là dữ liệu sẽ thay đổi inplace
df_oos = df_oos[df_oos['Date'].isin(from_to_date)]

In [ ]:
df_oos

In [ ]:
# Kiểm tra xem những store nào không nhập stock, những store nào duplicated thì không tính

if len(df_gps) != len(df_oos):
    # Merge DataFrames with indicator=True
    df_gps_to_check_stock = df_gps.drop_duplicates('Apple ID', keep='first')

    merged_df = df_gps_to_check_stock.merge(df_oos, on=['Staff Name', 'Apple ID', 'Channel', 'Store Name', 'Date'], 
                      how='left', indicator=True)
    
    # Filter rows that are only in df1 but not in df2
    df_gps_only = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
    df_gps_only =  df_gps_only[['Staff Name', 'Apple ID', 'Channel', 'Store Name', 'Date']]
df_gps_only.reset_index()

In [ ]:
# Xem 5 dòng đầu tiên của dữ liệu
df_oos.head()

In [ ]:
df_oos_dropped_duplicates =  df_oos.drop_duplicates('Apple ID', keep='first')

In [ ]:
len(df_oos_dropped_duplicates)

In [ ]:
# Define the column to start iterating after
start_col = 'AW'

# Find the index of the column to start from
start_index = df_oos_dropped_duplicates.columns.get_loc(start_col) + 1

# Iterate through the columns after the specified column
counts_dict = {}
for col in df_oos_dropped_duplicates.columns[start_index:]:
    try:
        count = (df_oos_dropped_duplicates[col] != 0).sum()
        counts_dict[col] = count
    except TypeError:
        print(df_oos_dropped_duplicates[col])

# Convert the results to a DataFrame
df = pd.DataFrame.from_dict(counts_dict, orient='index', columns=['Non-zero Count'])
df['Percentage'] = round(df['Non-zero Count'] / len(df_oos_dropped_duplicates) * 100, 2)

# Reset index to have the column names as a column
df.reset_index(inplace=True)
df.rename(columns={'index': 'Column Name'}, inplace=True)
df.sort_values('Percentage', inplace=True)
print(df)

In [ ]:
def update_graph(selected_range):
    # Filter the DataFrame based on the selected range
    if selected_range == '< 25%':
        filtered_df = df[df['Percentage'] < 25]
    elif selected_range == '25-50%':
        filtered_df = df[(df['Percentage'] >= 25) & (df['Percentage'] <= 50)]
    elif selected_range == '50-75%':
        filtered_df = df[(df['Percentage'] > 50) & (df['Percentage'] <= 75)]
    elif selected_range == '> 75%':
        filtered_df = df[df['Percentage'] > 75]
    
    # Create the bar chart
    fig = go.Figure(data=[go.Bar(
        x=filtered_df['Column Name'],
        y=filtered_df['Non-zero Count'],
        marker_color=filtered_df['Percentage'],
        text=filtered_df['Percentage'].round(2),
        textposition='auto',
    )])
    
    fig.update_layout(
        title='Phân loại mức độ tồn kho',
        xaxis_title='Product',
        yaxis_title='Count',
        coloraxis_colorbar=dict(title='Percentage'),
    )
    
    fig.show()

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=[
        ('< 25%', '< 25%'),
        ('25-50%', '25-50%'),
        ('50-75%', '50-75%'),
        ('> 75%', '> 75%')
    ],
    value='< 25%',
    description='Percentage Range:',
    style={'description_width': 'initial'}
)

# Display the interactive widget
interact(update_graph, selected_range=dropdown)